### 腳本結構

```
Train_main.py
├── Update_action_npy.py
├── Update_train.py
├── Generate_training_data.py
├── AugmentData.py
└── TrainEnsembleModel.py
```

### 運行範例

在命令行中運行這個腳本，您可以按照以下步驟進行：

1. **打開命令行**：根據您的操作系統，打開命令行工具（如Windows的CMD、MacOS或Linux的終端）。

2. **運行腳本**：

    - 使用默認的 `time_step` 值（70）：

        ```sh
        python Train_main.py
        ```

    - 使用自定義的 `time_step` 值（例如 100）：

        ```sh
        python Train_main.py --time_step=100
        ```

### 運行示例解釋

- 當您運行 `python Train_main.py` 時，腳本將按順序執行列表中的每個腳本。默認情況下，`time_step` 的值為 70。
- 如果您指定了 `--time_step` 參數，腳本將使用您提供的值來運行需要 `time_step` 的腳本。例如，`python Train_main.py --time_step=100` 將使用 `time_step=100` 來運行 `Update_train.py` 和 `TrainEnsembleModel.py`。

# 大致流程

### Update_action_npy.py
- **功能**：處理指定目錄中的視頻文件，使用 MediaPipe Pose 模型提取每幀的骨架關鍵點資訊，並對這些關鍵點以髖部中心進行標準化處理後保存為 `.npy` 文件。僅在視頻文件比 `.npy` 文件更新時重新處理視頻，並顯示處理進度和每幀視頻的平均推理時間。

### Update_train.py
- **功能**：清空指定目錄，並將處理過的數據片段（骨架關鍵點資訊）進行滑動窗口處理，提取固定幀數的片段，保存為新的 `.npy` 文件。

### Generate_training_data.py
- **功能**：合併訓練資料夾中的數據片段，並保存為一個 `.npz` 文件，用於後續的訓練。打印各類別的數據片段數量。

### AugmentData.py
- **功能**：對數據進行水平翻轉增強，將原始數據和翻轉後的數據合併，並保存為新的 `.npz` 文件。打印合併後數據的每個標籤數量。

### TrainEnsembleModel.py
- **功能**：訓練和評估姿態集成模型。包括數據加載和標籤編碼，數據集分割，模型初始化，訓練和評估模型，最後保存訓練好的模型。


# Update_action_npy.py

這段程式碼的功能是遍歷指定目錄中的視頻文件，使用 MediaPipe Pose 模型提取每幀的骨架關鍵點資訊，並對這些關鍵點以髖部中心進行標準化處理後保存為 .npy 文件。如果目錄中已有相應的 .npy 文件，則僅在視頻文件比 .npy 文件更新時重新處理視頻。程式會生成進度條顯示處理進度，並最終計算和顯示每幀視頻的平均推理時間。這樣的處理有助於生成標準化的骨架數據，便於後續的姿態分析和機器學習模型訓練。


1. **初始化**
    - 初始化 MediaPipe Pose 模型，用於後續骨架關鍵點檢測。

2. **定義路徑**
    - 設置輸入目錄（`action`）和輸出目錄（`action_npy`）。
    - 如果輸出目錄不存在，則創建該目錄。

3. **處理視頻的 `process_video` 函數**
    - 開啟視頻文件並初始化相關計時和數據變數。
    - 循環讀取視頻的每幀：
        - 計時讀取幀的時間。
        - 將 BGR 圖像轉換為 RGB。
        - 使用 MediaPipe Pose 模型進行姿態檢測。
        - 計算推理時間並累積。
        - 如果檢測到姿態，則提取關鍵點座標，並以左髖部和右髖部的中點為中心進行標準化。
        - 保存標準化後的骨架關鍵點數據到 `frames_data`。
    - 釋放視頻資源並將 `frames_data` 保存為 .npy 文件。
    - 返回每幀的平均推理時間。

4. **檢查視頻文件是否更新的 `is_new_file` 函數**
    - 檢查 .npy 文件是否存在。
    - 如果 .npy 文件存在，對比視頻文件和 .npy 文件的修改時間，判斷是否需要重新處理視頻文件。

5. **主程序邏輯**
    - 使用 `rich.progress` 顯示進度條。
    - 遍歷輸入目錄中的每個子目錄（每個動作類別）：
        - 對每個視頻文件，檢查相應的 .npy 文件是否需要更新。
        - 如果需要更新，將視頻文件和輸出文件路徑添加到待處理任務列表中。
    - 如果有待處理任務：
        - 初始化進度條任務。
        - 逐個處理待處理任務，對每個視頻文件調用 `process_video` 函數，並更新進度條。
        - 計算並顯示所有視頻文件的平均關鍵點數據，並生成相應的 .npy 文件，便於後續的數據分析和機器學習應用。

# Update_train.py

### 1. 初始化與設定
1. **設定資料夾路徑**
    - 定義原始數據目錄 `npy_path` 為 `"action_npy"`。
    - 定義輸出數據目錄 `train_path` 為 `"train"`。

### 2. 定義與清空資料夾的函數
2. **定義 `clear_directory` 函數**
    - 檢查指定的目錄是否存在。
    - 嘗試刪除該目錄及其內容。
    - 如果遇到許可權問題，使用系統命令強制刪除目錄。
    - 捕獲並打印其他可能的錯誤。

### 3. 定義滑動窗口函數
3. **定義 `sliding_window` 函數**
    - 接受數據 `data`、窗口大小 `window_size`、步長 `step_size` 和標籤 `label`。
    - 計算數據的總幀數、關鍵點數和坐標數。
    - 使用滑動窗口從數據中提取片段，每個片段包含 `window_size` 幀。
    - 將每個片段及其標籤添加到結果列表 `slices` 中。
    - 處理不足一個步長的剩餘片段，確保不漏掉數據。
    - 返回提取的數據片段列表。

### 4. 處理和儲存數據的函數
4. **定義 `process_and_save_data` 函數**
    - 接受原始數據目錄 `npy_path`、保存處理後數據的目錄 `train_path`、窗口大小 `window_size` 和步長 `step_size`。
    - 獲取所有動作類別（子目錄）的名稱。
    - 對每個動作類別，檢查並創建對應的保存目錄。
    - 遍歷每個動作類別下的所有 `.npy` 文件：
        - 加載數據文件。
        - 使用滑動窗口提取數據片段。
        - 將提取的片段保存為新的 `.npy` 文件，文件名中包含片段的索引。

### 5. 主程序邏輯
5. **主程序執行邏輯**
    - 使用 `argparse` 解析命令行參數，獲取滑動窗口大小 `time_step`。
    - 調用 `clear_directory` 函數清空 `train` 目錄。
    - 調用 `process_and_save_data` 函數處理原始數據，並將處理後的數據保存到 `train` 目錄。

### 完整流程
1. 初始化與設定資料夾路徑。
2. 定義清空資料夾的 `clear_directory` 函數。
3. 定義使用滑動窗口提取數據片段的 `sliding_window` 函數。
4. 定義處理和保存數據的 `process_and_save_data` 函數。
5. 在主程序中，解析命令行參數，清空 `train` 資料夾，並調用處理和保存數據的函數。

# Generate_training_data.py
當然，以下是這段程式碼的詳細流程條列說明：

### 1. 初始化與設定
1. **設定訓練資料夾路徑**
    - 定義訓練數據目錄 `train_path` 為 `"train"`。

### 2. 定義數據合併函數
2. **定義 `combine_data` 函數**
    - **初始化數據和標籤列表**
        - 初始化 `data_slices` 列表用於存儲數據片段。
        - 初始化 `labels` 列表用於存儲標籤。
    - **初始化標籤計數字典**
        - 初始化 `label_counts` 字典用於計數各類別的數據片段數量。
    - **讀取訓練資料夾中的所有子資料夾**
        - 使用 `os.listdir(train_path)` 讀取 `train_path` 資料夾中的所有子資料夾名稱（每個子資料夾代表一個動作類別）。
    - **遍歷每個子資料夾**
        - 檢查子資料夾是否為目錄。
        - 讀取子資料夾中的所有文件。
        - **遍歷每個文件**
            - 構建文件的完整路徑。
            - 加載 `.npy` 文件中的數據。
            - 將數據片段添加到 `data_slices` 列表中。
            - 將對應的標籤添加到 `labels` 列表中。
            - 更新 `label_counts` 字典中該類別的計數。
    - **將數據和標籤轉換為 numpy 數組**
        - 使用 `np.array(data_slices)` 將數據片段列表轉換為 numpy 數組。
        - 使用 `np.array(labels)` 將標籤列表轉換為 numpy 數組。
    - **保存數據和標籤到 `.npz` 文件**
        - 使用 `np.savez('data_combined.npz', data=data_slices, labels=labels)` 將數據和標籤保存到 `.npz` 文件。
    - **打印各類別的資料數量**
        - 遍歷 `label_counts` 字典並打印每個類別的數據片段數量。
        - 打印總的數據片段數量。

### 3. 主程序邏輯
3. **主程序執行邏輯**
    - 檢查是否作為主程序運行。
    - 調用 `combine_data` 函數執行數據合併並保存處理過的數據。

### 總結流程
1. 設定訓練資料夾路徑。
2. 定義 `combine_data` 函數，詳細步驟如下：
    - 初始化數據和標籤列表。
    - 初始化標籤計數字典。
    - 讀取訓練資料夾中的所有子資料夾。
    - 遍歷每個子資料夾，檢查是否為目錄。
    - 讀取子資料夾中的所有文件，遍歷每個文件。
    - 加載 `.npy` 文件中的數據並更新列表和字典。
    - 將數據和標籤轉換為 numpy 數組。
    - 保存數據和標籤到 `.npz` 文件。
    - 打印各類別的資料數量和總數量。
3. 在主程序中調用 `combine_data` 函數，執行數據合併並保存處理過的數據。


# AugmentData.py
以下是這段程式碼的條列流程說明：

### 1. 定義水平翻轉函數
1. **定義 `horizontal_flip` 函數**
    - 接受一個骨架數據（`skeleton`）作為輸入。
    - 複製骨架數據，創建翻轉後的骨架數據副本。
    - 將所有 x 座標（假設在偶數索引位置）取負數，實現水平翻轉。
    - 返回翻轉後的骨架數據。

### 2. 讀取數據
2. **讀取保存的數據**
    - 使用 `np.load('data_combined.npz')` 讀取之前保存的 `.npz` 文件。
    - 將數據片段提取到 `X` 中，將標籤提取到 `y` 中。

### 3. 對數據進行水平翻轉
3. **應用水平翻轉**
    - 使用 `horizontal_flip` 函數對 `X` 中的每一幀數據應用水平翻轉。
    - 將翻轉後的數據存儲到 `X_flipped` 中。
    - 複製標籤 `y` 到 `y_flipped`，標籤保持不變。

### 4. 合併原始數據和翻轉數據
4. **合併數據**
    - 使用 `np.concatenate` 將原始數據 `X` 和翻轉數據 `X_flipped` 合併。
    - 使用 `np.concatenate` 將原始標籤 `y` 和翻轉標籤 `y_flipped` 合併。
    - 合併後的數據存儲在 `X_combined` 中，合併後的標籤存儲在 `y_combined` 中。

### 5. 保存處理後的數據
5. **保存合併後的數據**
    - 使用 `np.savez('data_combined.npz', data=X_combined, labels=y_combined)` 將合併後的數據和標籤保存到 `.npz` 文件中。

### 6. 計算並打印各類別數量
6. **計算和打印標籤的數量**
    - 使用 `Counter` 計算每個標籤出現的次數。
    - 打印每個標籤對應的數據片段數量。

### 總結流程
1. 定義水平翻轉函數 `horizontal_flip`，對骨架數據進行水平翻轉。
2. 讀取保存的數據和標籤。
3. 對每幀數據應用水平翻轉，並保持標籤不變。
4. 合併原始數據和翻轉數據，以及對應的標籤。
5. 保存合併後的數據和標籤到 `.npz` 文件中。
6. 計算並打印合併後數據的每個標籤數量。


# TrainEnsembleModel.py
### 1. 設定與導入
1. **導入相關模組**
    - `os`：操作系統相關功能。
    - `numpy`：數值計算庫。
    - `torch`：PyTorch深度學習框架。
    - `torch.utils.data.Dataset, DataLoader`：PyTorch數據集和數據加載器。
    - `sklearn.model_selection.train_test_split`：數據集分割函數。
    - `sklearn.preprocessing.LabelEncoder`：標籤編碼器。
    - `torch.optim`：優化器。
    - `rich.progress`：進度條顯示工具。
    - `pickle`：數據序列化和反序列化。
    - `Posture_ensemble_model`：自定義模型類。
    - `torch.nn`：PyTorch神經網絡模組。
    - `argparse`：命令行參數解析器。

### 2. 主函數 `main`
2. **定義 `main` 函數**
    - **加載數據**
        - 從 `data_combined.npz` 文件中加載數據片段和標籤。
    - **標籤編碼**
        - 使用 `LabelEncoder` 將標籤轉換為整數表示。
        - 保存 `LabelEncoder` 到 `Model/posture_label_encoder.pkl` 文件。
        - 打印標籤列表。
    - **數據集分割**
        - 使用 `train_test_split` 將數據集分割為訓練集和測試集。
    - **創建數據集和數據加載器**
        - 定義 `SkeletonDataset` 類，用於加載數據片段和標籤。
        - 創建訓練和測試數據集。
        - 創建訓練和測試數據加載器。
    - **初始化模型和參數**
        - 設置輸入維度、隱藏層維度、層數、類別數和 TCN 通道數。
        - 初始化 LSTM、GRU 和 TCN 模型。
        - 創建集成模型。
        - 將模型移動到 GPU（如果可用）。
    - **訓練和評估模型**
        - 定義 `train_model` 函數，用於訓練模型。
        - 定義 `evaluate_model` 函數，用於評估模型。
        - 訓練模型。
        - 評估模型。
    - **保存模型**
        - 將訓練好的模型保存到 `Model/Posture_ensemble_model.pth` 文件。

### 3. 定義數據集類 `SkeletonDataset`
3. **定義 `SkeletonDataset` 類**
    - 初始化數據和標籤。
    - 返回數據集的長度。
    - 返回指定索引處的數據和標籤。

### 4. 訓練函數 `train_model`
4. **定義 `train_model` 函數**
    - 設置進度條。
    - 進行訓練過程：
        - 將模型設置為訓練模式。
        - 遍歷訓練數據加載器，進行前向傳播、計算損失、反向傳播和優化。
        - 更新進度條顯示訓練過程中的損失值和進度。

### 5. 評估函數 `evaluate_model`
5. **定義 `evaluate_model` 函數**
    - 設置進度條。
    - 進行評估過程：
        - 將模型設置為評估模式。
        - 遍歷測試數據加載器，進行前向傳播，計算準確率，並更新進度條顯示評估過程中的進度。
    - 打印模型在測試集上的準確率。

### 6. 主程序邏輯
6. **主程序執行邏輯**
    - 使用 `argparse` 解析命令行參數，設置滑動窗口的大小（`time_step`）。
    - 調用 `main` 函數，執行訓練和評估過程。

### 總結流程
1. 導入相關模組。
2. 定義主函數 `main`：
    - 加載數據並進行標籤編碼。
    - 分割數據集為訓練集和測試集。
    - 創建數據集和數據加載器。
    - 初始化模型和參數。
    - 訓練和評估模型。
    - 保存模型。
3. 定義數據集類 `SkeletonDataset`。
4. 定義訓練函數 `train_model`。
5. 定義評估函數 `evaluate_model`。
6. 使用 `argparse` 解析命令行參數，設置滑動窗口大小，並調用 `main` 函數。